# Laboratorio 2: Data Understanding

**Universidad del Valle de Guatemala**  
**Facultad de Ingeniería**  
**Departamento de Ciencias de la Computación**  
**Machine Learning Operations** 

## Integrantes

- Arturo Argueta - 21527 
- Edwin de León - 22809 
- Diego Leiva - 21752 
- Pablo Orellana - 21970

## Librerías

In [1]:
from pathlib import Path
import time
import pandas as pd
from tqdm import tqdm

## Lectura de datos

Durante el análisis exploratorio se encontró que hay conjuntos de datos con codificaciones diferentes a la típica utf-8, por lo que se necesita una lectura segura de datos

In [2]:
def read_csv_with_fallback(path, encodings=("utf-8", "latin1", "cp1252"), **pd_kwargs):
    """
    Intenta leer un CSV probando varias codificaciones para encontrar la correcta.

    Args:
        path (str): Ruta al archivo CSV.
        encodings (tuple): Codificaciones a probar.

    Returns:
        pd.DataFrame: DataFrame leído.
    """
    last_err = None

    # Itera sobre las codificaciones
    for enc in encodings:
        try:
            df = pd.read_csv(path, encoding=enc, **pd_kwargs)
            return df, enc
        except UnicodeDecodeError as e:
            last_err = e
        except Exception as e:
            # Otros errores; guarda y sigue probando por si es solo encoding
            last_err = e

    # Último intento “tolerante”: utf-8 con reemplazo para caracteres malos
    try:
        with open(path, "r", encoding="utf-8", errors="replace") as f:
            df = pd.read_csv(f, **pd_kwargs)
        return df, "utf-8(errors=replace)"
    except Exception:
        raise last_err

In [3]:
def load_csvs(
    folder: str,
    pattern: str = "*.csv",
    encodings=("utf-8", "latin1", "cp1252"),
    **pd_kwargs
):
    """
    Lee todos los CSVs que hagan match con `pattern` en `folder`,
    mostrando progreso con tqdm y midiendo el tiempo por archivo.

    Args:
        folder (str): Carpeta donde buscar los archivos CSV.
        pattern (str): Patrón de búsqueda para los archivos CSV.
        encodings (tuple): Codificaciones a probar al leer los CSVs.
        **pd_kwargs: Argumentos adicionales para pd.read_csv.

    Returns:
        tuple: (dfs, report) donde dfs es un diccionario de DataFrames y report es un DataFrame con el resumen de la carga.
    """
    # Obtiene la lista de archivos CSV
    files = sorted(Path(folder).glob(pattern))
    dfs = {}
    rows = []

    # Itera sobre los archivos CSV
    for p in tqdm(files, desc="Leyendo CSVs", unit="archivo"):
        t0 = time.perf_counter()  # Marca el tiempo de inicio
        status = "ok"  # Estado inicial
        used_enc = None  # Codificación utilizada
        nrows = ncols = None  # Filas y columnas
        err_msg = ""  # Mensaje de error
        try:
            # Intenta leer el CSV
            df, used_enc = read_csv_with_fallback(p, encodings=encodings, **pd_kwargs)
            dfs[p.name.split(".")[0]] = df
            nrows, ncols = df.shape
        except Exception as e:
            # Si falla, captura el error
            status = "error"
            err_msg = f"{type(e).__name__}: {e}"
        t1 = time.perf_counter()
        rows.append(
            {
                "archivo": p.name,
                "estado": status,
                "encoding": used_enc,
                "filas": nrows,
                "columnas": ncols,
                "tiempo_s": round(t1 - t0, 3),
                "ruta": str(p),
                "error": err_msg,
            }
        )
    report = pd.DataFrame(rows).sort_values(["estado", "archivo"])
    return dfs, report

In [4]:
# Lee todos los CSV de la carpeta "data", separador por coma (ajusta sep si necesitas ; o \t)
dfs, reporte = load_csvs("data/raw", sep=",")
display(reporte)


Leyendo CSVs: 100%|██████████| 5/5 [00:00<00:00,  7.90archivo/s]


,archivo,estado,encoding,filas,columnas,tiempo_s,ruta,error
0,categoria.csv,ok,utf-8,101,2,0.002,data\raw\categoria.csv,
1,cliente.csv,ok,latin1,12000,12,0.038,data\raw\cliente.csv,
2,evento.csv,ok,utf-8,2756101,5,0.585,data\raw\evento.csv,
3,marca.csv,ok,utf-8,307,2,0.001,data\raw\marca.csv,
4,producto.csv,ok,utf-8,12026,6,0.007,data\raw\producto.csv,


## Limpieza

### Manejo de duplicados

Durante el análisis exploratorio se encontró que los conjuntos de datos de clientes y eventos tienen registros completos duplicados.

In [5]:
def treat_duplicates(df, nombre_tabla):
    """
    Trata los duplicados en un DataFrame. 

    Args:
        df (pd.DataFrame): El DataFrame a procesar.
        nombre_tabla (str): El nombre de la tabla (o DataFrame) para el reporte.

    Returns:
        pd.DataFrame: El DataFrame sin duplicados.
    """
    original = len(df)  # Registros originales
    df_sin_dups = df.drop_duplicates()  # Elimina duplicados
    nuevo = len(df_sin_dups)  # Registros después de eliminar duplicados
    eliminados = original - nuevo  # Registros eliminados
    pct_conservado = (nuevo / original) * 100 # % Conservado

    print(f"Tabla: {nombre_tabla}")
    print(f" - Registros originales: {original}")
    print(f" - Registros después de eliminar duplicados: {nuevo}")
    print(f" - Duplicados eliminados: {eliminados}")
    print(f" - % Conservado: {pct_conservado:.2f}%\n")
    
    return df_sin_dups

In [6]:
# Tratamiento de duplicados para clientes y eventos
cliente = treat_duplicates(dfs["cliente"], "cliente")
evento = treat_duplicates(dfs["evento"], "evento")

# Sustituir en el diccionario
dfs["cliente"] = cliente
dfs["evento"] = evento

Tabla: cliente
 - Registros originales: 12000
 - Registros después de eliminar duplicados: 11720
 - Duplicados eliminados: 280
 - % Conservado: 97.67%

Tabla: evento
 - Registros originales: 2756101
 - Registros después de eliminar duplicados: 2755641
 - Duplicados eliminados: 460
 - % Conservado: 99.98%



### Manejo de nulos

Durante el análisis exploratorio se encontró que el conjunto de datos de  `clientes` tiene 281 registros nulos en cada columna (2.34%). en `eventos` solo la variable `transactionid` tiene 99.9% de valores nulos, y para `producto` las variables `categoria_id` y `marca_id` tienen 8.55% y 7.28% respectivamente y `precio` tiene un 0.05% de nulos.

Para tratarlos se decide eliminar los resgistros con datos nulos para `clientes` y la variable `precio`, para las variables `categoria_id` y `marca_id` se decide mapearlos como desconocido u otros. 

In [7]:
def next_free_id(series):
    """
    Obtiene el siguiente ID libre (no usado) a partir de una serie de IDs existentes.

    Args:
        series (pd.Series): Serie de IDs existentes.

    Returns:
        int: Siguiente ID libre.
    """
    if series.empty:
        return 1
    return int(pd.to_numeric(series, errors="coerce").max()) + 1

In [8]:
def simple_nulls_pipeline(dfs):
    """
    Aplica:
      - cliente: dropna(any)
      - evento: drop column 'transactionid'
      - producto: dropna(subset=['precio'])
      - categoria/marca: agrega 'Otro' con ID nuevo; mapea nulos de categoria_id/marca_id en producto a esos nuevos IDs.

    Args:
        dfs (dict): Diccionario de DataFrames por tabla.

    Returns:
        tuple: (dfs_actualizado, reporte_df, info_ids)
    """
    reportes = []
    out = {}

    # --- CATEGORIA: agregar "Otro" con ID nuevo ---
    categoria = dfs["categoria"].copy()
    nuevo_cat_id = next_free_id(categoria["id"])
    if not (categoria["id"] == nuevo_cat_id).any():
        categoria = pd.concat([
            categoria,
            pd.DataFrame([{"id": nuevo_cat_id, "categoria": "Otro"}])
        ], ignore_index=True)
    out["categoria"] = categoria

    # --- MARCA: agregar "Otro" con ID nuevo ---
    marca = dfs["marca"].copy()
    nuevo_marca_id = next_free_id(marca["id"])
    if not (marca["id"] == nuevo_marca_id).any():
        marca = pd.concat([
            marca,
            pd.DataFrame([{"id": nuevo_marca_id, "marca": "Otro"}])
        ], ignore_index=True)
    out["marca"] = marca

    # --- CLIENTE: eliminar filas con nulos ---
    cliente = dfs["cliente"]
    orig = len(cliente)
    cliente_clean = cliente.dropna(how="any")
    out["cliente"] = cliente_clean
    reportes.append({
        "tabla": "cliente",
        "accion": "Eliminar filas con nulos",
        "registros_originales": orig,
        "registros_resultantes": len(cliente_clean),
        "eliminados": orig - len(cliente_clean),
        "imputados": 0,
        "%_conservado": round(len(cliente_clean) / orig * 100, 2)
    })

    # --- EVENTO: quitar columna transactionid ---
    evento = dfs["evento"].copy()
    tenia_col = "transactionid" in evento.columns
    if tenia_col:
        evento = evento.drop(columns=["transactionid"])
    out["evento"] = evento
    reportes.append({
        "tabla": "evento",
        "accion": "Eliminar columna transactionid" if tenia_col else "Sin cambios",
        "registros_originales": dfs["evento"].shape[0],
        "registros_resultantes": evento.shape[0],
        "eliminados": 0,
        "imputados": 0,
        "%_conservado": 100.0
    })

    # --- PRODUCTO: eliminar sin precio + mapear nulos de categoria_id/marca_id a 'Otro' ---
    producto = dfs["producto"].copy()
    orig_prod = len(producto)

    # quitar sin precio
    sin_precio = producto["precio"].isna().sum()
    producto = producto.dropna(subset=["precio"])

    # mapear nulos de categoria_id/marca_id a los nuevos IDs
    imput_cat = producto["categoria_id"].isna().sum() if "categoria_id" in producto.columns else 0
    imput_marca = producto["marca_id"].isna().sum() if "marca_id" in producto.columns else 0

    if "categoria_id" in producto.columns:
        producto["categoria_id"] = producto["categoria_id"].fillna(nuevo_cat_id).astype("Int64")
    if "marca_id" in producto.columns:
        producto["marca_id"] = producto["marca_id"].fillna(nuevo_marca_id).astype("Int64")

    out["producto"] = producto
    reportes.append({
        "tabla": "producto",
        "accion": "Eliminar sin precio; mapear nulos de categoria_id/marca_id a 'Otro'",
        "registros_originales": orig_prod,
        "registros_resultantes": len(producto),
        "eliminados": sin_precio,
        "imputados": int(imput_cat + imput_marca),
        "%_conservado": round(len(producto) / orig_prod * 100, 2)
    })

    # --- INFO de las claves nuevas para que las uses en joins y labels ---
    info_ids = {
        "nuevo_categoria_id": nuevo_cat_id,
        "nuevo_marca_id": nuevo_marca_id
    }

    reporte_df = pd.DataFrame(reportes, columns=[
        "tabla","accion","registros_originales","registros_resultantes",
        "eliminados","imputados","%_conservado"
    ])

    return out, reporte_df, info_ids

In [9]:
dfs_limpio, reporte, info = simple_nulls_pipeline(dfs)
# Imprime los nuevos IDs generados
print("Nuevos IDs generados:")
for key, value in info.items():
    print(f"- {key}: {value}")
display(reporte) # resumen de lo conservado/eliminado/imputado

Nuevos IDs generados:
- nuevo_categoria_id: 102
- nuevo_marca_id: 308


,tabla,accion,registros_originales,registros_resultantes,eliminados,imputados,%_conservado
0,cliente,Eliminar filas con nulos,11720,11719,1,0,99.99
1,evento,Eliminar columna transactionid,2755641,2755641,0,0,100.00
2,producto,Eliminar sin precio; mapear nulos de categoria...,12026,12020,6,1904,99.95
